<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2023notebooks/2023_0126ccap_tmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## はじめに

符号化器-復号化器モデルは，[seq2seq](https://arxiv.org/abs/1409.3215) と呼ばれるモデルでもある。
邦訳すれば，系列-2-系列 モデルなのだが，今回のプロジェクトでは，意味表象が，系列ではない。
そのため，seq2seq という名称よりも，より一般化した，符号化器-復号化器 (enc-dec model) モデルとしたい。

<center>
<img src="https://raw.githubusercontent.com/komazawa-deep-learning/komazawa-deep-learning.github.io/master/assets/2014Sutskever_S22_Fig1.svg" width="66%">
<br/>    
From Sutskever 2014, Figure 1.
</center>

上図は，seq2seq モデルの概略図である。
符号化器と呼ばれる部分は，トークン `<EOS>` が入力された時点までである。
それ以降は，復号化器となる。
符号化器と復号化器とで，一時刻前の中間層の状態が共有されていることがポイントである。
seq2seq は翻訳モデルであり，符号化器と復号化器とで，言語モデルの扱う言語が異なっている。
具体的には，フランス語と英語である。


オリジナルの，三角モデルにおける o2p については，三層のニューラルネットワークとみなしうる。
このため，o2p の中間層は，識別性能を向上する役割と，モダリティ間の結合という２つの異なる役割を担っていたとみなすことができる。
理論的には，両者を分離する必要も，統合する必要もない，どちらにしても積極的な理由は存在しないと思われる。
駄菓子菓子，計算論的な役割においては，異なるモダリティ間の通信を媒介する役割と，入力モダリティにおける表象を確立するという意味合いを分離すると，役割分担が明確になるのであろうということである。

---

* 一文字の orth2phon を担保したいために，全角の数字，アルファベット，ひらがな，計 109 文字をデータ先頭に追加した。
* Fushimi1999 (Psyc. Rev.) の語彙リストを fushimi1999_list として収録
* Fushimi1999_list の扱いに伴い訓練語彙数を 10K から 20K に増加
* 学習率 lr は 0.001 だと収束しない。0.0001 であれば良好であり，訓練損失 0.01 程度，訓練精度 0.987 程度までに至る。
* ただし，一文字データセット onechar_dataset では lr=0.001 の方が収束が早い。
これは，データセットサイズが 20K と 0.1K と 20 倍の差があるためであろう。
* 近藤先生が，GPU 上で実行してくださった訓練済モデルのファイル名が `decoder256new.pt` と `encoder256new.pt` である。
これは，中間層ユニット数が 256 である orth2phon モデルの訓練済モデルである。
* `_train()` 関数内で，正解判定をする際に，GPU から CPU へ転送しなければいけないことを忘れていたので修正した。
具体的には， `detach()` と `numpy()` の間に `cpu()` を挿入した。2 箇所
```python
    ok_flag = (ok_flag) and (decoder_output.argmax() == target_tensor[di].detach().cpu().numpy()[0])
```
* 近藤先生の GPU で訓練済モデルを CPU 環境で実行する必要がある場合，変更して読み込む必要がある

```python
encoder_pretrained_fname = 'encoder256new.pt'
decoder_pretrained_fname = 'decoder256new.pt'
if os.path.exists(encoder_pretrained_fname):
    encoder = torch.load(encoder_pretrained_fname, map_location=torch.device(device))
    
if os.path.exists(decoder_pretrained_fname):
    decoder = torch.load(decoder_pretrained_fname, map_location=torch.device(device))
```

近藤先生の実験によれば，結果は以下の通りである(そうだ)。

正答率

|   | 条件 | 記述         | 正解率 | 
|:----|:-----|:------------|:------|
|WORD |   HF |1:consistent |　18/20
|WORD |   HF |2:typical    |   HF___inconsist  16/20|
|WORD |   HF |3:atypical   |   HF___atypical_  8/20 |
|WORD |   LF |1:consistent |   LF___consist__  14/20|
|WORD |   LF |2:typical    |   LF___inconsist  9/20|
|WORD |   LF |3:atypical   |   LF___atypical_  3/20|

* 伏見らではでなかったatypical効果だけでなく，
　consistent-typicalの差もある程度ある気がします
 また，LFでも効果ありであり，かつ，頻度効果もあり
* **今回，L(legitimate alternative reading of components） マークを付けてみました**
  Lm, Lnは，モーラ間違い，一文字間違いと混合


アクセプト率

|     | 条件 | 記述         | 正解率 | 
|:----|:----|:------------|:------|
|非単語| HF  | 1:consistent|HFNW_consist__  17/20|
|非単語| HF  | 2:typical   |HFNW_inconsist　　17/20|
|非単語| HF  | 3:ambiguous |HFNW_ambiguous  13/20|
|非単語| LF  | 1:consistent|LFNW_consist__  15/20|
|非単語| LF  | 2:typical   |LFNW_inconsist  13/20|
|非単語| LF  | 3:ambiguous |LFNW_ambiguous  7/20|

* かなり読めますね．アクセプトは，どんな読みでもいいので読めそうな読み方ならOKにしています．
　単語の L と同じになります．
* **結構驚きは，非単語のときに連濁や促音化ができているところ**


In [ ]:
# ここはお遊びなので，スキップしても良い
import IPython
#IPython.display.Image(url="https://livedoor.blogimg.jp/ftb001/imgs/b/4/b4629a79.jpg")
#IPython.display.Image(url="https://uy-allstars.com/_assets/images/pages/char/detail/webp/lum@pc.webp")

import os
lum_img_fname = 'lum@pc.webp'
if not os.path.exists(lum_img_fname):
    !wget "https://uy-allstars.com/_assets/images/pages/char/detail/webp/lum@pc.webp"


import matplotlib.pyplot as plt
x = plt.imread('lum@pc.webp')
plt.figure(figsize=(5,8))
plt.axis('off')
plt.imshow(x)

## 0.1 結果の描画

In [ ]:
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

#import matplotlib    
#matplotlib.rcParams['text.usetex'] = True    
fig, ax = plt.subplots(1,2, figsize=(8,4))

fig.suptitle('Fushimi+1999 単語リストの検証 (o2p, hid256)')
ax[0].plot((18/20,16/20, 8/20), marker="v", color="green", label="高頻度")
ax[0].plot((14/20, 9/20, 3/20), marker="^", color="blue", label="低頻度")
ax[0].set_xlim(-0.5,2.5)
ax[0].set_ylim(0,1)
ax[0].set_xticks(ticks=range(3))
ax[0].set_xticklabels(labels=['一貫','非一貫','例外'])
ax[0].legend()
ax[0].set_title('単語')
ax[0].set_ylabel('正解率')

ax[1].plot((17/20,17/20,13/20), marker="v", color="green", label="高頻度")
ax[1].plot((15/20,13/20, 7/20), marker="^", color="blue", label="低頻度")
ax[1].set_xlim(-0.5,2.5)
ax[1].set_ylim(0,1)
ax[1].set_xticks(ticks=range(3))
ax[1].set_xticklabels(labels=['一貫','非一貫','例外'])
ax[1].set_title('非単語')
ax[1].legend()
fig.savefig('2023_0123LAM_o2p_hid256_fushimi1999.pdf')
plt.show()

# 1 準備作業

## 1.1 ライブラリのインポート

In [ ]:
%config InlineBackend.figure_format = 'retina'

import torch
from IPython import get_ipython
import os
isColab =  'google.colab' in str(get_ipython())
HOME = os.environ['HOME']

if isColab:
    # `import bit` する前に termcolor を downgrade しないと colab ではテキストに色がつかない
    !pip install --upgrade termcolor==2.0 2>&1  
    import termcolor    

    # !pip install ipynbname --upgrade > /dev/null 2>&1 
    # !git clone https://github.com/ShinAsakawa/bit.git 2>&1


if isColab:
    # colab 上で MeCab を動作させるために，C コンパイラを起動して，MeCab の構築を行う
    # そのため時間がかかる。
    !apt install aptitude
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
    !pip install mecab-python3==0.7
    !pip install jaconv
    
    import MeCab
    wakati = MeCab.Tagger('-Owakati').parse
    yomi = MeCab.Tagger('-Oyomi').parse
    
else:
    from ccap.mecab_settings import yomi
    from ccap.mecab_settings import wakati

# if isColab:
#     !pip install jupyter_contrib_nbextensions 2>&1 
#     !jupyter nbextension enable codefolding/main 2>&1

## 1.2 パラメータ設定

語彙数を 10K 語から 20K 語に倍増しているのは，Fushimi1999 の語彙リストの未知語が存在したためである。

In [ ]:
if isColab:
    # psylex71utf8.txt.gz, ntt_psylex.py, fushimi1999.py, triangle2_utils.py, 2023_0126triangle2.pt,
    from google.colab import files
    _ = files.upload()


In [ ]:
if isColab:
    !mkdir ccap
    !mv psylex71utf8.txt.gz ccap

In [ ]:
%reload_ext autoreload
%autoreload 2

from triangle2_utils import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


X = torch.load('2023_0126triangle2.pt', map_location=torch.device(device))
encoder = X['encoder']
decoder = X['decoder']
params  = X['params']

# モデルの概要を印字
print(f'encoder:{encoder}')
print(f'decoder:{decoder}')
#print(f'params:{params}')
for k, v in params.items():
    print(f'{k}:{colored(v,"red",attrs=["bold"])}')

params['loss_func'] = torch.nn.NLLLoss()    

In [ ]:
# シミュレーションに必要なパラメータの設定
try:
    params
except:
    params = {
        'traindata_size':  20000,   # 訓練データ数，NTT 日本語語彙特性の高頻度語を上位から抽出
        #'traindata_size': 301612,  # 訓練データ数，NTT 日本語語彙特性の高頻度語を上位から抽出
        'epochs': 30,               # 学習のためのエポック数
        #'hidden_size': 256,          # 中間層のニューロン数
        'hidden_size': 64,         # 中間層のニューロン数
        'random_seed': 42,          # 乱数の種。ダグラス・アダムス著「銀河ヒッチハイカーズガイド」

        # 以下 `source` と `target` を定義することで，別の課題を実行可能
        'source': 'orth',          # ['orth', 'phon', 'mora', 'mora_p', 'mora_p_r']
        'target': 'orth',          # ['orth', 'phon', 'mora', 'mora_p', 'mora_p_r']
        #'target': 'mora_p_r',     # ['orth', 'phon', 'mora', 'mora_p', 'mora_p_r']
        # 'orth': 書記素, 
        # 'phon': 音韻, 
        # 'mora': モーラ
        # 'mora_p': モーラを silius による音分解
        # 'mora_p_r': モーラの silius 音分解の逆
        'pretrained': False,          # True であれば訓練済ファイルを読み込む
        #'pretrained': True,          # True であれば訓練済ファイルを読み込む
        #'isTrain'   : True,          # True であれば学習する
    
        # 学習済のモデルパラメータを保存するファイル名
        #'path_saved': '2022_0607lam_o2p_hid32_vocab10k.pt', 
        #'path_saved': '2022_0829lam_p2p_hid24_vocab10k.pt',
        'path_saved': False,                      # 保存しない場合
    
        # 結果の散布図を保存するファイル名    
        'path_graph': '2023_0115lam_p2o_hid32_vocabntt_freq20k.pdf',
        'path_graph': False,                     # 保存しない場合

        'lr': 1e-4,                     # 学習率
        'dropout_p': 0.0,                 # ドロップアウト率
        'teacher_forcing_ratio': 0.5,     # 教師強制を行う確率
        'optim_func': torch.optim.Adam,   # 最適化アルゴリズム ['torch.optim.Adam', 'torch.optim.SGD', 'torch.optim.AdamW']
        'loss_func' :torch.nn.NLLLoss(),  # 負の対数尤度損失 ['torch.nn.NLLLoss()', or 'torch.nn.CrossEntropyLoss()']
}

In [ ]:
from fushimi1999 import fushimi1999
from fushimi1999 import _fushimi1999_list
fushimi1999_list = _fushimi1999_list()

In [ ]:
%reload_ext autoreload
%autoreload 2

import ntt_psylex

_vocab = ntt_psylex.psylex71_dataset(
    traindata_size=params['traindata_size'],
    w2v=None,
    yomi=yomi,
    stop_list=fushimi1999_list,
    source=params['source'],
    target=params['target'],
)

top_n = 300
print(f'語彙先頭の項目 {top_n} を印字')
for i, wrd in enumerate(_vocab.word_list[:top_n]):
    _end = " " if (i+1) % 10 != 0 else "\n"
    print((i+1, wrd), end=_end)

In [ ]:
import math
import random
import numpy as np
import time

from triangle2_utils import convert_ids2tensor    
from triangle2_utils import calc_accuracy    
from triangle2_utils import asMinutes
from triangle2_utils import timeSince
from triangle2_utils import check_vals_performance
from triangle2_utils import evaluate
from triangle2_utils import _train
from triangle2_utils import _fit
#from triangle2_utils import evaluate
from triangle2_utils import Onechar_dataset

In [ ]:
def calc_accuracy(
    _dataset,
    encoder,
    decoder,
    max_length=None,
    source_vocab=None,
    target_vocab=None,
    source_ids=None,
    target_ids=None,
    isPrint=False):

    ok_count = 0
    for i in range(_dataset.__len__()):
        _input_ids, _target_ids = _dataset.__getitem__(i)
        _output_words, _output_ids, _attentions = evaluate(
            encoder=encoder,
            decoder=decoder,
            input_ids=_input_ids,
            max_length=max_length,
            source_vocab=source_vocab,
            target_vocab=target_vocab,
            source_ids=source_ids,
            target_ids=target_ids,
        )
        ok_count += 1 if _target_ids == _output_ids else 0
        if (_target_ids != _output_ids) and (isPrint):
            print(i, _target_ids == _output_ids, _output_words, _input_ids, _target_ids)

    return ok_count/_dataset.__len__()

In [ ]:
onechar_dataset = Onechar_dataset(
    source=params['source'], 
    target=params['target'],
    yomi=yomi,
    _vocab=_vocab)

In [ ]:
try:
    losses
except:
    losses = []
losses += _fit(encoder=encoder, 
               decoder=decoder, 
               device=device,
               epochs=10,
               #epochs=params['epochs'], 
               max_length=_vocab.source_maxlen,
               n_sample=0,
               params=params,
               source_vocab=_vocab.source_vocab,
               target_vocab=_vocab.target_vocab,
               source_ids=_vocab.source_ids,
               target_ids=_vocab.target_ids,
               teacher_forcing_ratio=params['teacher_forcing_ratio'],
               #train_dataset=train_dataset,
               train_dataset=onechar_dataset,
               #lr=params['lr'],
               lr=0.001,
               val_dataset=None,
               #val_dataset=X_vals,
              )

In [ ]:
plt.plot(losses)

In [ ]:
N = 3000
_orth_ids = _vocab.__getitem__(N)[0]
_phon_ids = _vocab.__getitem__(N)[1]
_orth = _vocab.orth_ids2tkn(_orth_ids)
_phon = _vocab.phon_ids2tkn(_phon_ids)
print(_orth, _phon)
# print(_vocab.orth_ids2tkn(_vocab.__getitem__(N)[0]))
# print(_vocab.phon_ids2tkn(_vocab.__getitem__(N)[1]))

In [ ]:
def check_fushimi1999_list(encoder=encoder,
                           decoder=decoder,
                           is_print:bool=False,
                          )->str:

    ret_msg = ""
    counter = 1
    for key, val in fushimi1999.items():
        key_old = ""
        if key != key_old:
            key_old = key
            ret_msg += f'{key}:'
            if is_print:
                print(colored(f'{key}:', 'green', attrs=['bold']), end=" ")
        
        n_ok, n_all = 0, 0
        msg = ""
        for wrd in val:
            _orth = _vocab.orth_tkn2ids(wrd) + [_vocab.orth_vocab.index('<EOW>')]
            ans=evaluate(encoder,
                         decoder,
                         _orth,
                         _vocab.source_maxlen,
                         _vocab.source_vocab,
                         _vocab.target_vocab,
                         _vocab.source_ids,
                         _vocab.target_ids)
            
            res = "".join(p for p in ans[0][:-1])  # モデルからの戻り値を再構成
            if res == wrd:
                n_ok += 1
            n_all += 1

            counter =  1 if (counter % 10) == 0 else (counter + 1)
            _end = "\n" if counter==1 else ", "
            # print(f'{wrd}',
            #       colored(f'/{res}/','grey', attrs=['bold']), 
            #       end=_end)
            msg += f"{wrd}->/{res}/{_end}"
            
        ret_msg += f'{n_ok/n_all * 100:5.2f}%\n{msg}'
        if is_print:
            print(f'{n_ok/n_all * 100:5.2f}%\n{msg}')
            
    return ret_msg
            
print(check_fushimi1999_list())

In [ ]:
P  = int(_vocab.__len__() * 0.9)
_P = _vocab.__len__() - P
train_dataset, val_dataset = torch.utils.data.random_split(dataset=_vocab,
                                                           lengths=(P, _P),
                                                           generator=torch.Generator().manual_seed(params['random_seed']))


In [ ]:
try:
    losses
except:
    losses = []
losses += _fit(encoder=encoder, 
               decoder=decoder, 
               device=device,
               #epochs=10,
               epochs=params['epochs'], 
               max_length=_vocab.source_maxlen,
               n_sample=0,
               params=params,
               source_vocab=_vocab.source_vocab,
               target_vocab=_vocab.target_vocab,
               source_ids=_vocab.source_ids,
               target_ids=_vocab.target_ids,
               teacher_forcing_ratio=params['teacher_forcing_ratio'],
               train_dataset=train_dataset,
               #train_dataset=onechar_dataset,
               lr=params['lr'],
               #lr=0.001,
               val_dataset={'val': val_dataset},
               #val_dataset=X_vals,
              )

In [ ]:
torch.save({'encoder':encoder, 'decoder':decoder, 'params':params, 'losses':losses}, '2023_0126triangle2.pt')
if isColab:
    files.download('2023_0126triangle2.pt')